<a href="https://colab.research.google.com/github/dakleshchevnikova/LSML_SGA/blob/main/LSML2_FP_Kleshchevnikova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final project
## Daria Kleshchevnikova
## Task definition

The input is a sentence and the output is an integer 0 or 1 that corresponds to a prediction wwhether the sentence is grammatically correct (1) or incorrect (0).

The dataset used for model training is the CoLA (The Corpus of Linguistic Acceptability).

The training data consists of texts (sentences, as discussed above), thus, it's embeddings are obtained (pre-trained BERT is used) before processing to the main model (SVC).

Model's accuracy is later tested on the in-domain dev dataset. With the obtained result of over 75%, we may conclude that the model was trained successfully.


In [1]:
!pip install wget --quiet

  Preparing metadata (setup.py) ... done


In [2]:
!pip install transformers --quiet

In [3]:
!pip install mlflow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 623.0/623.0 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.1 MB/s eta 0:00:00


In [4]:
import wget
import os
import random
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
import logging
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score



In [5]:
url = 'https://nyu-mll.github.io/CoLA/cola_public_1.1.zip'

if not os.path.exists('./cola_public_1.1.zip'):
    wget.download(url, './cola_public_1.1.zip')

if not os.path.exists('./cola_public/'):
    !unzip cola_public_1.1.zip

Archive:  cola_public_1.1.zip
   creating: cola_public/
  inflating: cola_public/README      
   creating: cola_public/tokenized/
  inflating: cola_public/tokenized/in_domain_dev.tsv  
  inflating: cola_public/tokenized/in_domain_train.tsv  
  inflating: cola_public/tokenized/out_of_domain_dev.tsv  
   creating: cola_public/raw/
  inflating: cola_public/raw/in_domain_dev.tsv  
  inflating: cola_public/raw/in_domain_train.tsv  
  inflating: cola_public/raw/out_of_domain_dev.tsv  


In [6]:
df = pd.read_csv("./cola_public/raw/in_domain_train.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])
df_dev = pd.read_csv("./cola_public/raw/in_domain_dev.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])

df.head(10)

,sentence_source,label,label_notes,sentence
0,gj04,1,NaN,"Our friends won't buy this analysis, let alone..."
1,gj04,1,NaN,One more pseudo generalization and I'm giving up.
2,gj04,1,NaN,One more pseudo generalization or I'm giving up.
3,gj04,1,NaN,"The more we study verbs, the crazier they get."
4,gj04,1,NaN,Day by day the facts are getting murkier.
5,gj04,1,NaN,I'll fix you a drink.
6,gj04,1,NaN,Fred watered the plants flat.
7,gj04,1,NaN,Bill coughed his way out of the restaurant.
8,gj04,1,NaN,We're dancing the night away.
9,gj04,1,NaN,Herman hammered the metal flat.


In [7]:
sentences = df.sentence.values
labels = df.label.values

sentences_dev = df_dev.sentence.values
labels_dev = df_dev.label.values


In [8]:
logging.basicConfig(level=logging.INFO)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model = model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [9]:
def get_sentence_embedding(text, tokenizer, model):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt').to(device)
    with torch.no_grad():
      sentence_embedding = model(**t).last_hidden_state
    return sentence_embedding[:, 0, :].cpu().numpy()

In [10]:
svmm = SVC(C=1.0, kernel='rbf', degree=3, gamma='scale', coef0=0.0, shrinking=True,
           probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False,
           max_iter=- 1, decision_function_shape='ovr', break_ties=False, random_state=0)

emb = [get_sentence_embedding(s, tokenizer, model)[0] for s in sentences]
emb_dev = [get_sentence_embedding(s, tokenizer, model)[0] for s in sentences_dev]

svmm.fit(emb, labels)
pr = svmm.predict(emb_dev)

acc = accuracy_score(labels_dev, pr)



In [11]:
print(f"Accuracy on the in-domain dev data: {acc * 100:.2f}%")

Accuracy on the in-domain dev data: 75.14%


Let us attempt to finetune the model's usind the mlflow tracking to keep track of the experiments.

In [12]:
import mlflow
import mlflow.sklearn
import warnings

In [13]:
experiment_name = 'experiment_1'
warnings.filterwarnings("ignore")

In [14]:
experiment = mlflow.create_experiment(experiment_name)


In [15]:
mlflow.set_experiment(experiment_name)

for c in (0.5, 1, 2, 3, 4, 5):
    with mlflow.start_run():
        svmm = SVC(C=c, kernel='rbf', degree=3, gamma='scale', coef0=0.0, shrinking=True,
                   probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False,
                   max_iter=- 1, decision_function_shape='ovr', break_ties=False, random_state=0)
        svmm.fit(emb, labels)
        pr = svmm.predict(emb_dev)
        acc = accuracy_score(labels_dev, pr)

        print(f"SVC model (C={c}):")
        print(f"Accuracy on the in-domain dev data: {acc * 100:.2f}%")

        mlflow.log_param("C", c)
        mlflow.log_metric("accuracy", acc)

        mlflow.sklearn.log_model(svmm, "model")

SVC model (C=0.5):
Accuracy on the in-domain dev data: 73.81%


2024/12/18 20:22:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


SVC model (C=1):
Accuracy on the in-domain dev data: 75.14%


2024/12/18 20:22:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


SVC model (C=2):
Accuracy on the in-domain dev data: 76.09%


...

2024/12/18 20:24:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


SVC model (C=4):
Accuracy on the in-domain dev data: 78.18%


2024/12/18 20:24:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


SVC model (C=5):
Accuracy on the in-domain dev data: 77.61%


2024/12/18 20:25:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [23]:
runs = mlflow.search_runs(experiment_ids=experiment)

In [24]:
runs

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.accuracy,params.C,tags.mlflow.log-model.history,tags.mlflow.source.name,tags.mlflow.user,tags.mlflow.runName,tags.mlflow.source.type
0,d4444e95c080420798f856cb2272596b,238758580068138269,FINISHED,file:///content/mlruns/238758580068138269/d444...,2024-12-18 20:24:34.414000+00:00,2024-12-18 20:25:12.960000+00:00,0.776091,5,"[{""run_id"": ""d4444e95c080420798f856cb2272596b""...",/usr/local/lib/python3.10/dist-packages/colab_...,root,languid-vole-397,LOCAL
1,8cb38382e96b4be0859db7d2ee989eed,238758580068138269,FINISHED,file:///content/mlruns/238758580068138269/8cb3...,2024-12-18 20:24:03.856000+00:00,2024-12-18 20:24:34.410000+00:00,0.781784,4,"[{""run_id"": ""8cb38382e96b4be0859db7d2ee989eed""...",/usr/local/lib/python3.10/dist-packages/colab_...,root,dashing-roo-939,LOCAL
2,4ebed1673d244acfb784d43030ec56b6,238758580068138269,FINISHED,file:///content/mlruns/238758580068138269/4ebe...,2024-12-18 20:23:22.333000+00:00,2024-12-18 20:24:03.853000+00:00,0.772296,3,"[{""run_id"": ""4ebed1673d244acfb784d43030ec56b6""...",/usr/local/lib/python3.10/dist-packages/colab_...,root,delicate-goat-224,LOCAL
3,39876894b935472ba24c28b991d9059a,238758580068138269,FINISHED,file:///content/mlruns/238758580068138269/3987...,2024-12-18 20:22:43.783000+00:00,2024-12-18 20:23:22.330000+00:00,0.760911,2,"[{""run_id"": ""39876894b935472ba24c28b991d9059a""...",/usr/local/lib/python3.10/dist-packages/colab_...,root,sassy-ray-706,LOCAL
4,d096baf9456844388255b91921b28a95,238758580068138269,FINISHED,file:///content/mlruns/238758580068138269/d096...,2024-12-18 20:22:08.511000+00:00,2024-12-18 20:22:43.779000+00:00,0.751423,1,"[{""run_id"": ""d096baf9456844388255b91921b28a95""...",/usr/local/lib/python3.10/dist-packages/colab_...,root,clean-bee-876,LOCAL
5,731f67e9ba1848889f900695b3406aea,238758580068138269,FINISHED,file:///content/mlruns/238758580068138269/731f...,2024-12-18 20:21:35.847000+00:00,2024-12-18 20:22:08.509000+00:00,0.738140,0.5,"[{""run_id"": ""731f67e9ba1848889f900695b3406aea""...",/usr/local/lib/python3.10/dist-packages/colab_...,root,illustrious-gull-620,LOCAL


In [25]:
max_id = ''
max_acc = 0
for id in runs.run_id:
  a = mlflow.get_run(run_id=id).data.metrics['accuracy']
  if a >= max_acc:
    max_acc = a
    max_id = id
  print('C: ', mlflow.get_run(run_id=id).data.params['C'], ', accuracy: ', a)

C:  5 , accuracy:  0.7760910815939279
C:  4 , accuracy:  0.7817836812144212
C:  3 , accuracy:  0.7722960151802657
C:  2 , accuracy:  0.7609108159392789
C:  1 , accuracy:  0.7514231499051234
C:  0.5 , accuracy:  0.7381404174573055


In [26]:
  print('Best run_id:', max_id)
  print('Best C: ', mlflow.get_run(run_id=max_id).data.params['C'], ', best accuracy: ', max_acc)

Best run_id: 8cb38382e96b4be0859db7d2ee989eed
Best C:  4 , best accuracy:  0.7817836812144212


We have successfully found that the best performing model was the one with parameter C = 4 and that it had provided accuracy just over 78%.


Note: the code above takes a few minutes to run on Google Colab T4 GPU (and significantly longer on cpu), the flask api consequently was not created. Below, an example of obtaining model predictions is provided. We can see that the model has managed to classify the sentences correctly (0 for the sentence with mistake, 1 otherwise).

In [27]:
svmm = SVC(C=4, kernel='rbf', degree=3, gamma='scale', coef0=0.0, shrinking=True,
           probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False,
           max_iter=- 1, decision_function_shape='ovr', break_ties=False, random_state=0)

svmm.fit(emb, labels)


SVC(C=4, random_state=0)

In [34]:
input_sentence_wrong = 'Task are finished.'

In [35]:
svmm.predict(get_sentence_embedding(input_sentence_wrong, tokenizer, model))

array([0])

In [36]:
input_sentence_cor_1 = 'Task is finished.'

In [37]:
svmm.predict(get_sentence_embedding(input_sentence_cor_1, tokenizer, model))

array([1])

In [38]:
input_sentence_cor_2 = 'Tasks are finished.'

In [39]:
svmm.predict(get_sentence_embedding(input_sentence_cor_2, tokenizer, model))

array([1])